In [1]:
import numpy as np
import pandas as pd

from datetime import datetime as dt
import random
import re

from tqdm import tqdm
import os


from itertools import chain
from collections import Counter

In [2]:
data = pd.read_pickle("../../Data/app_opp.pickle")
data.sample(4)

,ApplicationId,OpportunityId,Title,ExternalDescription,WhatYouDid,dateAppliedAt,JobTitle,LicenseAndCertification,skill,major,degree
9131,aYBDY2gv9k2G7WJFPYc6Ow==,LIxNNHPOuUmhOSQSWg5MSA==,"RN - Labor & Delivery - Full Time, Nights **$5...",<h2>The Labor &amp; Delivery RN is a a clinica...,Responsible for basic care of residents and as...,2020-08-26 17:01:33.422,Certified Nursing Assistant,Registered Nurse,Testing,Nursing,Associate of Science
17639,AeXJD9GbPEWBU30EJ1Wdqw==,QQnYmmjNqU6JD6HjRyWbXA==,Project Coordinator,<p>High Liner Foods is looking for a Project C...,• Working as a Senior Project Manager leading ...,2020-12-02 22:32:33.703,Senior Project Manager at CIBC,Certification in Business Process Reengineerin...,Enterprise resource planning,Mechanical Engineering,Bachelor of Engineering
13181,w7568h7NEEei5t+6ii0lwQ==,zIi0WAMVMEm7dctwt9J9UQ==,Employee Health Clinic Nurse Manager,<p>Manages the daily activities of three netwo...,None,2020-02-06 15:01:11.908,Practice Coordinator,CPR certified,HIPAA,None,On job Training
49887,vIMeeUaRMkSMFXFG3tmWiA==,yWgqsBogkEOuTYIsvNdlJA==,"ACNO Laredo Medical Center - Laredo, TX",<p><em>Laredo Medical Center is Laredo&rsquo;s...,•\tInitiated processes and implemented to meet...,2022-10-05 07:20:01.253,"Director ED, Trauma, CAART, Audit and Billing",RN,Recruiting,Nursing,Bachelor of Science in Nursing


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   ApplicationId            70000 non-null  object        
 1   OpportunityId            70000 non-null  object        
 2   Title                    70000 non-null  object        
 3   ExternalDescription      70000 non-null  object        
 4   WhatYouDid               65755 non-null  object        
 5   dateAppliedAt            70000 non-null  datetime64[ns]
 6   JobTitle                 70000 non-null  object        
 7   LicenseAndCertification  70000 non-null  object        
 8   skill                    70000 non-null  object        
 9   major                    43059 non-null  object        
 10  degree                   70000 non-null  object        
dtypes: datetime64[ns](1), object(10)
memory usage: 5.9+ MB


In [4]:
applicant_columns = ['ApplicationId', 'dateAppliedAt', 'JobTitle', "WhatYouDid", "LicenseAndCertification",
                     "major", "degree", "skill"]

opportunity_columns = ['OpportunityId', 'Title', 'ExternalDescription']

In [5]:
#Get a random sample application using the ApplicationId
sampleApplication = data.sample(1)['ApplicationId'].values[0]


sample_df = data.loc[data['ApplicationId'] == sampleApplication]

LicenseAndCertification_values = sample_df.LicenseAndCertification.unique()
skill_values = sample_df.skill.unique()
major_values = sample_df.major.unique()
degree_values = sample_df.degree.unique()

application_aggregated_info = pd.DataFrame()

application_aggregated_info['skill_values'] = [skill_values]
application_aggregated_info['major_values'] = [major_values]
application_aggregated_info['LicenseAndCertification_values'] = [LicenseAndCertification_values]
application_aggregated_info['degree_values'] = [degree_values]

application_aggregated_info

,skill_values,major_values,LicenseAndCertification_values,degree_values
0,"[Microsoft Office, Microsoft SQL Server, Medic...",[None],"[NextGen Certified Professional-EHR, NextGen C...","[National, On job Training]"


In [6]:
# Create a DataFrame with aggregated application information
application_aggregated_info = pd.DataFrame()

applications = data.ApplicationId.unique()

for sampleApplication in tqdm(applications, desc='Processing applications', unit='application'):
    sample_df = data.loc[data['ApplicationId'] == sampleApplication]

    Opp_Id = sample_df['OpportunityId'].unique()[0]
    Opp_Title = sample_df['Title'].unique()[0]
    Opp_ExternalDescription = sample_df['ExternalDescription'].unique()[0]
    
    WhatYouDid_values = sample_df['WhatYouDid'].unique()
    JobTitle_values = sample_df['JobTitle'].unique()
    skill_values = sample_df['skill'].unique()
    major_values = sample_df['major'].unique()
    LicenseAndCertification_values = sample_df['LicenseAndCertification'].unique()
    degree_values = sample_df['degree'].unique()

    temp_df = pd.DataFrame({'ApplicationId': [sampleApplication],
                            'OpportunityId': Opp_Id,
                            'Title': Opp_Title,
                            'ExternalDescription': [Opp_ExternalDescription],
                            'WhatYouDid_values': [WhatYouDid_values],
                            'JobTitle_values': [JobTitle_values],
                            'skill_values': [skill_values],
                            'major_values': [major_values],
                            'LicenseAndCertification_values': [LicenseAndCertification_values],
                            'degree_values': [degree_values]})

    application_aggregated_info = pd.concat([application_aggregated_info, temp_df], ignore_index=True)

Processing applications: 100%|██████████| 175/175 [00:01<00:00, 139.67application/s]


In [7]:
# Assuming you are in the Notebooks folder
# Move up one level to Talent_Alignment_Lib and then go to the Data folder
data_folder = "../../Data"

# Define the filename for the pickle file
pickle_filename = "consolidated_applications.pkl"

# Full path to the pickle file in the Data folder
pickle_filepath = os.path.join(data_folder, pickle_filename)

# Assuming you have the DataFrame `application_aggregated_info` ready
# Save the DataFrame as a pickle file
application_aggregated_info.to_pickle(pickle_filepath)

NameError: name 'os' is not defined